# PCA and APT with the Euro Stoxx 50

## Load Libraries and Data

In [ ]:
# Load required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from datetime import datetime

# Load scikit-learn library (LinearRegression only)
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA

In [ ]:
# Load data
close_prices = pd.read_csv('EUROSTOXX50DataV2.csv', index_col = 0)
Symbols = close_prices.columns

# Number of calendar days between first and last data points in close_prices
first_date = datetime.strptime(close_prices.index.min(), '%Y-%m-%d')
last_date = datetime.strptime(close_prices.index.max(), '%Y-%m-%d')
n_cal_days = (last_date - first_date).days

## Principal Component Analysis

In this section, we conduct a principal component analysis of the daily (annualized) returns of the constitutent stocks of the Euro Stoxx 50 to identify appropriate factors for a $k$-factor model of the returns of the constitutent stocks.

In [ ]:
## - DATA PREPARATION

# Annualization factor
ann_factor = 1 / ((n_cal_days / 365) / len(close_prices))

# Calculate percentage returns
daily_ret = close_prices / close_prices.shift(1) - 1
daily_ret = daily_ret.dropna()

# Annualize the observed daily returns
daily_ret = ann_factor * daily_ret

# Remove the index from the analysis
daily_ret_stocks = daily_ret.iloc[:, 1:len(Symbols)]

# Constructed centered data set
c_daily_ret_stocks = daily_ret_stocks - daily_ret_stocks.mean(axis = 0)

# Estimate the covariance matrix of the centered data set, return as an array
cov_c_ret = c_daily_ret_stocks.cov().to_numpy()

First, we find a singular value decomposition (SVD) $\hat{\Sigma} = U D^2 U^\top$ of the (estimated) covariance matrix $\hat{\Sigma}$ of the centered data set. The goal of this part is to determine the appropriate number of principal components.

**Note:** We will use the ``sklearn.decomposition.PCA`` class to compute the principal components and the transformed data, rather than manually using the SVD of $\hat{\Sigma}$.

In [ ]:
## - SVD of the covariance matrix of the centered data set

# SVD using np.linalg.svd
U, D2, V = np.linalg.svd(cov_c_ret)

# Determine total variance/inertia of sample
inertia = sum(D2)

# Determine cumulative variance measured by first k principal components
cumulative_var = np.cumsum(D2 / inertia)
cumulative_var

Suppose we are happy that the first $k=5$ principal components account for slightly over 65\% of the variability in the data. In the next part, we will transform/project the data along these principal components. 

We will conduct three runs of the PCA:

1. PCA with two principal components for visualization
2. PCA with three principal components for visualization
3. PCA with five principal components

In [ ]:
## - PCA with 2 principal components

# PCA with sklearn.decomposition.PCA
pca = PCA(n_components = 2)
pca_factors = pca.fit_transform(c_daily_ret_stocks)

# Visualize PCs
fig, ax = plt.subplots()
plt.scatter(pca_factors[:, 0], pca_factors[:, 1], alpha = 0.5)
plt.xlabel('1st PC')
plt.ylabel('2nd PC')

# # Covariance of PCs
# pca_factors.T @ pca_factors     # Observe that it is approximately diagonal (as expected)

In [ ]:
## - PCA with 3 principal components

# PCA with sklearn.decomposition.PCA
pca = PCA(n_components = 3)
pca_factors = pca.fit_transform(c_daily_ret_stocks)

# Visualize PCs in three dimensions
import mpl_toolkits.mplot3d  # noqa: F401

fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d", elev=-150, azim=110)

ax.scatter(
    pca_factors[:, 0],
    pca_factors[:, 1],
    pca_factors[:, 2],
    s=40,
)

ax.set_title("First three PCA dimensions")
ax.set_xlabel("1st PC")
ax.xaxis.set_ticklabels([])
ax.set_ylabel("2nd PC")
ax.yaxis.set_ticklabels([])
ax.set_zlabel("3rd PC")
ax.zaxis.set_ticklabels([])

plt.show()

In [ ]:
## - PCA with 5 principal components

# PCA with sklearn.decomposition.PCA
pca = PCA(n_components = 5)

# Determine the principal components u_1,...,u_k (columns of U_k), given as transpose(U_k)
pca_comps = pca.fit(c_daily_ret_stocks).components_

# Apply the transform to the centered daily returns
pca_factors = pca.fit_transform(c_daily_ret_stocks)

# Covariance of PCs
pca_cov = np.cov(pca_factors, rowvar = False)     # Observe that it is approximately diagonal (as expected)
pca_cov_inv = np.linalg.inv(pca_cov)

# # The following should yield the same output as pca_factors
# (pca_comps @ c_daily_ret_stocks.T).T

# # Should be the same as the kth entry in cumulative_var above
# sum(pca.explained_variance_ratio_)

## Five-Factor Model with the Principal Components

The PCA implemented above leads to a $k$-factor model 

$$R(t) = A + B F(t) + \epsilon(t),$$

where $A = \mathsf{E}[R]$, $B = U_k$ (the matrix whose columns are the first $k$ principal components), $F = U_k^\top x(t)$ (where $x(t) = R(t) - M$ is the centered return at time $t$), and $\epsilon(t) = \sum_{j=k+1}^d (u_j^\top x(t))u_j$ (where $u_{k+1},\dots,u_d$ are the remaining columns of $U$ not included in $U_k$). 

In [ ]:
## - Coefficients of k-factor model with PCA factors

# B matrix
kfm_B = pca_comps.T

# A vector
kfm_A = daily_ret_stocks.mean(axis = 0)

In [ ]:
## - Compare returns covariance matrices estimated directly from data and from the k-factor model 

# Returns estimated using exact k-factor model \hat{R} = A + BF
est_ret_stocks = (kfm_A.to_numpy().reshape((-1,1)) + kfm_B @ pca_factors.T).T

# Residuals of the exact k-factor model (R - \hat{R})
kfm_residuals = daily_ret_stocks - est_ret_stocks

# Covariance matrix of returns implied by k-factor model \tilde{R} = A + BF + \epsilon (see Slide 20)
kfm_ret_cov = kfm_B @ pca_cov @ kfm_B.T + kfm_residuals.cov()

# Compare with empirical estimate of returns covariance matrix
kfm_ret_cov - daily_ret_stocks.cov()     # Should be equal to 0 with some rounding error/tolerance

Since the mean and covariance matrix of returns calculated from the $k$-factor model agree with the mean and covariance matrix estimated directly from the data, we would not expect any drastic differences in the outcome of the mean-variance analysis when we use $M$ and $\Sigma$ estimated directly from the data or when we use the estimates from the $k$-factor model.

## Risk Premium Estimation via Arbitrage Pricing Theory

We wish to find $\lambda_0 \in \mathbb{R}$ and $\lambda \in \mathbb{R}^k$ such that the APT condition holds:

$$\mathsf{E}[R] = \lambda_0 \mathbf{1}_d + B \lambda.$$

Since by construction of the PCA $k$-factor model, we have $\mathsf{E}[F] = \mathbf{0}_5$ and $\mathsf{E}[\epsilon] = \mathbf{0}_d$, the APT condition can be written as

$$A = \lambda_0 \mathbf{1}_d + B \lambda.$$

The required $\lambda_0,\lambda$ can be estimated by solving the minimization problem 

\begin{equation*}
\min_{\lambda_0 \in \mathbb{R}, \lambda \in \mathbb{R}^k} \|A - \lambda_0 \mathbf{1}_d - B \lambda\|^2,
\end{equation*}

which is equivalent to a statistical regression of $A$ against $B$ with an intercept. If the minimum value attained in the above optimization is not equal to 0, then the APT condition does not hold.

In [ ]:
## - Approximate the risk premium

# Estimation via linear regression
reg = LinearRegression(fit_intercept=True).fit(kfm_B, kfm_A)
apt_lambda = reg.coef_
apt_lambda0 = reg.intercept_

# Determine if minimum is 0
vec1 = np.linspace(1, 1, len(kfm_A))
obj_func_value = sum((kfm_A - apt_lambda0 * vec1 - kfm_B @ apt_lambda) ** 2)
print("Minimum value:", obj_func_value)
print("Optimal lambda:", apt_lambda)
print("Optimal lambda_0:", apt_lambda0)

Since a minimum of zero is not achieved, the APT condition does not hold and the $k$-factor model based on the principal components is not an APT model. Indeed, a risk-free investment portfolio can be constructed such that its return is not equal to $\lambda_0$ (this is an arbitrage opportunity).

In [ ]:
## - Construct risk-free investment portfolio

# Load null_space command
from scipy.linalg import null_space

# Determine an orthonormal basis for Null(B^T)
basis_nsBt = null_space(kfm_B.T)

# Any linear combination yields a risk-free investment portfolio (appropriately scaled)
rf_investment = basis_nsBt.sum(axis = 1) / basis_nsBt.sum()

# Determine expected return of risk-free investment portfolio
mean_ret = daily_ret_stocks.mean(axis = 0)
mean_rf_investment = rf_investment.T @ kfm_A

mean_rf_investment

In this code, setting $k = 50$ yields a minimum value of $0$ (within tolerance) for the minimization problem for finding $\lambda_0, \lambda_1$. In this case, the resulting PCA $k$-factor model is an APT model and so a risk-free investment strategy cannot be constructed.